In [ ]:
#reload source files automatically
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from event_clustering.preprocessing import *

# Loading Preprocessing Analyzing

In [ ]:
file_name = 'DomesticDeclarations'
df = preprocess(load('data/' + file_name + '.xes'))
original_df_columns = df.columns

In [ ]:
analyze(df)
df.head(10)

In [ ]:
# Todo column name mapping

# Feature Generation

In [ ]:
add_event_type_representative(df)

In [ ]:
add_event_ref(df, 1)
add_event_ref(df, -1)
determine_event_position(df)

In [ ]:
add_timestamp_features(df)

In [ ]:
df

In [ ]:
df.to_csv('data/' + file_name +'_features.csv', index=False)

# Encoding

In [ ]:
df = preprocess(pd.read_csv('data/' + file_name +'_features.csv'))

In [ ]:
df_name = tfidf_encode(df, 'concept:name', TfidfVectorizer(stop_words = 'english'))
df_role = one_hot_encode(df, 'org:role')
df_event_position = df[['feature_pos_beginning', 'feature_pos_middle', 'feature_pos_end']]
df_time = binning(df, 'event_ref_time_1', 10)
df_time_of_day = df[['feature_time_00-06', 'feature_time_07-12', 'feature_time_13-18', 'feature_time_19-24']]

In [ ]:
df_encoded = df_name
df_encoded = df_encoded.join(df_role)
df_encoded = df_encoded.join(df_event_position)
df_encoded = df_encoded.join(df_time)
df_encoded = df_encoded.join(df['feature_day_nr'])

In [ ]:
# file_name_structure: [Dataset-name]_[Feature-groups]_encoded.csv
df_encoded.to_csv('data/' + file_name +'_name-role-pos-ref-tod_encoded.csv', index=False)

In [ ]:
df_encoded

In [ ]:
df_encoded.columns